In [14]:
import torch
import transformers
from transformers import AutoModelForMaskedLM, AutoTokenizer, pipeline
import gensim
import numpy as np
import json

In [8]:
print(torch.cuda.is_available())

True


In [9]:
def load_models(base_model_path, trained_model_path_1, trained_model_path_2):
    base_model = AutoModelForMaskedLM.from_pretrained(base_model_path, torch_dtype=torch.float16, device_map="auto")
    trained_model_1 = AutoModelForMaskedLM.from_pretrained(trained_model_path_1, torch_dtype=torch.float16, device_map="auto")
    trained_model_2 = AutoModelForMaskedLM.from_pretrained(trained_model_path_2, torch_dtype=torch.float16, device_map="auto")
    
    tokenizer = AutoTokenizer.from_pretrained(base_model_path)
    
    base_pipe = pipeline(
        "fill-mask", 
        model=base_model, 
        tokenizer=tokenizer,
        torch_dtype=torch.float16, 
        device_map="auto"
    )
    print(base_pipe)

    trained_pipe_1 = pipeline(
        "fill-mask", 
        model=trained_model_1, 
        tokenizer=tokenizer,
        torch_dtype=torch.float16, 
        device_map="auto"
    )

    trained_pipe_2 = pipeline(
        "fill-mask", 
        model=trained_model_2, 
        tokenizer=tokenizer,
        torch_dtype=torch.float16, 
        device_map="auto"
    )

    return base_pipe, trained_pipe_1, trained_pipe_2

def get_word_embedding(word, model):
    if word in model:
        return model[word]
    else:
        print(f"'{word}' not found in vocabulary. Returning zero vector.")
        return np.zeros(300)

def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    if norm1 == 0 or norm2 == 0:
        return 0.0
    return dot_product / (norm1 * norm2)

def choose_response(category_list, response_list, model):
    results = []

    all_categories = set(cat for categories in category_list for cat in categories)
    category_embeddings = {category: get_word_embedding(category, model) for category in all_categories}

    for categories, responses in zip(category_list, response_list):
        predict_dict = {word: 0 for word in categories}

        for response in responses:  
            cur_embedding = get_word_embedding(response['token_str'].strip(), model)
            cur_weight = response['score']

            print(f"Processing: '{response['token_str']}', Score: {cur_weight}")
            for category in categories: 
                cat_embd = category_embeddings[category]
                sim = cosine_similarity(cat_embd, cur_embedding)
                print(f"Category: '{category}', Similarity: {sim:.4f}")
                predict_dict[category] += cur_weight * sim
        
        best_category = max(predict_dict, key=predict_dict.get)
        results.append((best_category, predict_dict))  
    return results


def kl_divergence(p, q):
    epsilon = 1e-10  # Small value to avoid log(0) or division by zero
    p_values = np.array(list(p.values())) + epsilon
    q_values = np.array(list(q.values())) + epsilon
    return np.sum(p_values * np.log(p_values / q_values))

def softmax(scores):
    exp_scores = np.exp(list(scores.values())) 
    sum_exp_scores = np.sum(exp_scores)        
    softmax_scores = {k: v / sum_exp_scores for k, v in zip(scores.keys(), exp_scores)}  # Normalize
    return softmax_scores

In [10]:
"""Main function to run the pipeline and process results."""
# Configuration
test_name = "test_3"
base_model_path = '/home/ujx4ab/ondemand/NLP_Final_Political_Bias_Shifts/src/Local Models/FacebookAI_roberta-large'
CNN_model_path = "/home/ujx4ab/ondemand/NLP_Final_Political_Bias_Shifts/fine_tuned_llms/FacebookAI_roberta-large/checkpoints/CNN_model_3_a_5_epoch/2024-12-15 22-17-39/checkpoint-6450"
Fox_model_path = "/home/ujx4ab/ondemand/NLP_Final_Political_Bias_Shifts/fine_tuned_llms/FacebookAI_roberta-large/checkpoints/Fox_model_3_a_5_epoch/2024-12-15 22-17-39/checkpoint-6600"
word_vec_path = '/home/ujx4ab/ondemand/NLP_Final_Political_Bias_Shifts/word2vec/GoogleNews-vectors-negative300.bin.gz'

categories = {
    "test_1": [
        ["good", "bad"],
        ["very", "somewhat", "not"],
        ["very", "somewhat", "not"]
    ],
    "test_3": [
        ["very", "somewhat", "not"],
        ["difficult", "easy"],
        ["very", "somewhat", "not"],
        ["poorly", "well"]
    ]
}

cloze_statements = {
    "test_1": [
        "During the presidential campaign, artificial intelligence (AI) will be used for <mask>",
        "I am <mask> concerned that artificial intelligence (AI) will be used to create and distribute fake information about the presidential candidates and campaigns",
        "I am <mask> confident technology companies like Facebook, X (formerly Twitter), TikTok and Google, to prevent the misuse of their platforms to influence the election."
    ],
    "test_3": [
        "I have seen inaccurate news about the 2024 presidential election <mask> often.",
        "I find it <mask> to determine what is true and what is not when getting news and information about the presidential campaign and candidates",
        "I have been following news about candidates for the presidential election <mask> closely",
        "The news media has covered the presidential election <mask>."
    ]
}

print("Loading model\n")
base_pipe, CNN_pipeline, Fox_pipeline = load_models(base_model_path, CNN_model_path, Fox_model_path)

print("Loading word2vec model\n")
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format(word_vec_path, binary=True)

print("Generating results\n")    
base_results = [base_pipe(statement) for statement in cloze_statements[test]]
CNN_results = [CNN_pipeline(statement) for statement in cloze_statements[test]]
Fox_results = [Fox_pipeline(statement) for statement in cloze_statements[test]]

print("Determining most likely response\n")
results_base_model = choose_response(categories[test], base_results, word2vec_model)
CNN_fine_tuned_model = choose_response(categories[test], CNN_results, word2vec_model)
Fox_fine_tuned_model = choose_response(categories[test], Fox_results, word2vec_model)



Loading model



Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0


Loading word2vec model

Generating results

Determining most likely response

Processing: ' too', Score: 0.33642578125
Category: 'very', Similarity: 0.6065
Category: 'somewhat', Similarity: 0.3555
Category: 'not', Similarity: 0.5153
Processing: ' quite', Score: 0.27880859375
Category: 'very', Similarity: 0.7689
Category: 'somewhat', Similarity: 0.5678
Category: 'not', Similarity: 0.4772
Processing: ' more', Score: 0.1126708984375
Category: 'very', Similarity: 0.3496
Category: 'somewhat', Similarity: 0.3340
Category: 'not', Similarity: 0.2271
Processing: ' very', Score: 0.07989501953125
Category: 'very', Similarity: 1.0000
Category: 'somewhat', Similarity: 0.5031
Category: 'not', Similarity: 0.3947
Processing: ' fairly', Score: 0.068359375
Category: 'very', Similarity: 0.6954
Category: 'somewhat', Similarity: 0.5276
Category: 'not', Similarity: 0.3593
Processing: ' difficult', Score: 0.5771484375
Category: 'difficult', Similarity: 1.0000
Category: 'easy', Similarity: 0.5891
Processing: 

In [13]:
for base, CNN, Fox in zip(results_base_model, CNN_fine_tuned_model, Fox_fine_tuned_model):
    print("base: ", base)
    print("CNN: ", CNN)
    print("Fox: ", Fox)

base:  ('very', {'very': 0.5852647459541913, 'somewhat': 0.39180519840010675, 'not': 0.3880963543051621})
CNN:  ('very', {'very': 0.637011854472803, 'somewhat': 0.39082881363719935, 'not': 0.4248174971071421})
Fox:  ('very', {'very': 0.570014864904806, 'somewhat': 0.3478782877427875, 'not': 0.41155074682319537})
base:  ('difficult', {'difficult': 0.8140547269522358, 'easy': 0.5224675089816628})
CNN:  ('difficult', {'difficult': 0.8006565802304522, 'easy': 0.5263499530306035})
Fox:  ('difficult', {'difficult': 0.8280949357731515, 'easy': 0.5375519666233686})
base:  ('very', {'very': 0.9195476235609021, 'somewhat': 0.48814092481734406, 'not': 0.38466160898587987})
CNN:  ('very', {'very': 0.9211490907073312, 'somewhat': 0.4780448556052761, 'not': 0.3786351143307911})
Fox:  ('very', {'very': 0.9187117364954247, 'somewhat': 0.4740024585573792, 'not': 0.3761548927891454})
base:  ('well', {'poorly': 0.2412988352557477, 'well': 0.2697112230325729})
CNN:  ('well', {'poorly': 0.20019026919544558

In [16]:
# Initialize an empty list to store results
results = []

# Iterate through the models
for base, CNN, Fox in zip(results_base_model, CNN_fine_tuned_model, Fox_fine_tuned_model):
    # Calculate softmax distributions
    softmax_base = softmax(base[1])
    softmax_CNN = softmax(CNN[1])
    softmax_Fox = softmax(Fox[1])

    # Calculate KL divergence scores
    kl_div_base_CNN = kl_divergence(softmax_base, softmax_CNN)
    kl_div_base_Fox = kl_divergence(softmax_base, softmax_Fox)
    kl_div_CNN_Fox = kl_divergence(softmax_CNN, softmax_Fox)

    # Print results
    print("base softmax: ", softmax_base)
    print("CNN softmax: ", softmax_CNN)
    print("Fox softmax: ", softmax_Fox)
    print("KL Divergence scores: ")
    print(f"\tBase-CNN: {kl_div_base_CNN}")
    print(f"\tBase-Fox: {kl_div_base_Fox}")
    print(f"\tCNN-Fox: {kl_div_CNN_Fox}")
    print()

    # Store the results in a dictionary
    results.append({
        "prediction": base[0],
        "softmax_base": softmax_base,
        "softmax_CNN": softmax_CNN,
        "softmax_Fox": softmax_Fox,
        "kl_divergence": {
            "base_CNN": kl_div_base_CNN,
            "base_Fox": kl_div_base_Fox,
            "CNN_Fox": kl_div_CNN_Fox
        }
    })

formatted_results = json.dumps(results, indent=4)
print("Formatted Results: ")
print(formatted_results)

with open(f"/home/ujx4ab/ondemand/NLP_Final_Political_Bias_Shifts/Results/{test}.json", "w") as f:
    f.write(formatted_results)


base softmax:  {'very': 0.3780496029282145, 'somewhat': 0.3115518771392319, 'not': 0.3103985199325536}
CNN softmax:  {'very': 0.38601295954192666, 'somewhat': 0.30177686962904215, 'not': 0.3122101708290312}
Fox softmax:  {'very': 0.3767529033729503, 'somewhat': 0.3017059799085498, 'not': 0.32154111671849994}
KL Divergence scores: 
	Base-CNN: 0.00024460966426517196
	Base-Fox: 0.0003565010210799472
	CNN-Fox: 0.00024960528361761547

base softmax:  {'difficult': 0.5723846656894457, 'easy': 0.4276153343105543}
CNN softmax:  {'difficult': 0.5681498686273262, 'easy': 0.4318501313726738}
Fox softmax:  {'difficult': 0.5721290555698056, 'easy': 0.4278709444301943}
KL Divergence scores: 
	Base-CNN: 3.65750874815737e-05
	Base-Fox: 1.3345692628216393e-07
	CNN-Fox: 3.2315982619669434e-05

base softmax:  {'very': 0.44736109574806765, 'somewhat': 0.2906033823386466, 'not': 0.26203552191328566}
CNN softmax:  {'very': 0.4497767259491069, 'somewhat': 0.2887747752752847, 'not': 0.26144849877560844}
Fox so